In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ipywidgets

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
import io


In [ ]:
# Define paths
base_dir = '/kaggle/input/random-images-for-face-emotion-recognition'  # Adjust this path if necessary
image_size = (48, 48)
batch_size = 32


In [ ]:
# Data preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.1,
    shear_range=0.1,
    rotation_range=10
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
validation_generator = validation_datagen.flow_from_directory(
    base_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
# Display a few images with their labels
def display_sample_images(generator, class_names):
    x, y = next(generator)
    plt.figure(figsize=(12, 12))
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.imshow(x[i])
        plt.title(class_names[np.argmax(y[i])])
        plt.axis('off')
    plt.show()

class_names = list(train_generator.class_indices.keys())
display_sample_images(train_generator, class_names)


In [ ]:
# Use transfer learning for feature extraction
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

model.summary()


In [ ]:
# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Setup callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_emotion_model.keras', save_best_only=True)


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[early_stopping, model_checkpoint]
)


In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')
plt.show()


In [ ]:
# Load the best model
best_model = tf.keras.models.load_model('best_emotion_model.keras')

# Evaluate the model on the validation set
val_loss, val_accuracy = best_model.evaluate(validation_generator)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate predictions
val_predictions = best_model.predict(validation_generator)
val_pred_classes = np.argmax(val_predictions, axis=1)
val_true_classes = validation_generator.classes

# Get the number of classes from class_names
num_classes = len(class_names)

# Classification report
print(classification_report(val_true_classes, val_pred_classes, labels=range(num_classes), target_names=class_names))

# Confusion matrix
conf_matrix = confusion_matrix(val_true_classes, val_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# ROC and AUC for each class
plt.figure(figsize=(12, 8))
for i, class_name in enumerate(class_names):
    fpr, tpr, _ = roc_curve(val_true_classes == i, val_predictions[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'{class_name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from PIL import UnidentifiedImageError

# Load the best model
best_model = tf.keras.models.load_model('best_emotion_model.keras')

# Define the path to the test images
test_dir = '/kaggle/input/happiness'

# Define image size and class names
image_size = (48, 48)
class_names = list(train_generator.class_indices.keys())

# Function to preprocess and predict emotions for images
def classify_emotions(test_dir, model):
    test_images = [os.path.join(test_dir, fname) for fname in os.listdir(test_dir)]
    predictions = []
    
    for img_path in test_images:
        try:
            img = image.load_img(img_path, target_size=image_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array /= 255.0  # Normalize the image
            
            prediction = model.predict(img_array)
            predicted_class = class_names[np.argmax(prediction)]
            predictions.append((img_path, img, predicted_class))
        except UnidentifiedImageError:
            print(f"Cannot identify image file {img_path}. Skipping...")
        except Exception as e:
            print(f"Error processing file {img_path}: {e}. Skipping...")
    
    return predictions

# Classify emotions for test images
test_predictions = classify_emotions(test_dir, best_model)

# Function to display images with predicted labels
def display_predictions(predictions):
    plt.figure(figsize=(15, 15))
    for i, (img_path, img, predicted_class) in enumerate(predictions):
        plt.subplot(5, 5, i + 1)
        plt.imshow(img)
        plt.title(predicted_class)
        plt.axis('off')
    plt.show()

# Print the predictions and display images with predicted labels
for img_path, img, predicted_class in test_predictions:
    print(f'Image: {img_path} - Predicted Emotion: {predicted_class}')

display_predictions(test_predictions)